https://github.com/owid/covid-19-data/blob/master/public/data/README.md

In [1]:
!pip install pyspark
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('data_processing').getOrCreate()

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

### Чтение данных из файла и их анализ                                                       

In [4]:
df=spark.read.csv("/content/drive/MyDrive/Netology/owid-covid-data.csv",header=True,inferSchema=True)

In [5]:
 # посмотрим схему 
 df.printSchema()

root
 |-- iso_code: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- location: string (nullable = true)
 |-- date: string (nullable = true)
 |-- total_cases: double (nullable = true)
 |-- new_cases: double (nullable = true)
 |-- new_cases_smoothed: double (nullable = true)
 |-- total_deaths: double (nullable = true)
 |-- new_deaths: double (nullable = true)
 |-- new_deaths_smoothed: double (nullable = true)
 |-- total_cases_per_million: double (nullable = true)
 |-- new_cases_per_million: double (nullable = true)
 |-- new_cases_smoothed_per_million: double (nullable = true)
 |-- total_deaths_per_million: double (nullable = true)
 |-- new_deaths_per_million: double (nullable = true)
 |-- new_deaths_smoothed_per_million: double (nullable = true)
 |-- reproduction_rate: double (nullable = true)
 |-- icu_patients: double (nullable = true)
 |-- icu_patients_per_million: double (nullable = true)
 |-- hosp_patients: double (nullable = true)
 |-- hosp_patients_per_million

In [6]:
df.show(15)

+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+-----------+------------------+----------+-------------+-------------+--------------+--

### Очистка данных

In [7]:
# Удаляем все строки с OWID, так как это, по всей видимости, агрегированные данные по регионам
df_clean = df.filter(~df['iso_code'].startswith('OWID'))

# изменяем null на 0
df_clean = df_clean.fillna({'total_cases': 0, 'new_cases': 0})
df_clean.show(5)

+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+-----------+------------------+----------+-------------+-------------+--------------+--

### Задние 1. Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших) 

In [8]:
# Считаем процент заболевших относительно популяции
df_clean = df_clean.withColumn("total_cases_per_population", df_clean['total_cases']/df_clean['population']) 
df_clean.show()

+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+-----------+------------------+----------+-------------+-------------+--------------+--

In [9]:
df_clean.select(['iso_code','location','date', 'total_cases_per_population']).filter(df['date']=='2020-03-31').orderBy('total_cases_per_population', ascending=False).show(15)

+--------+-------------+----------+--------------------------+
|iso_code|     location|      date|total_cases_per_population|
+--------+-------------+----------+--------------------------+
|     VAT|      Vatican|2020-03-31|      0.007416563658838072|
|     SMR|   San Marino|2020-03-31|      0.006953857033413872|
|     AND|      Andorra|2020-03-31|      0.004866368989840161|
|     LUX|   Luxembourg|2020-03-31|      0.003479366621084...|
|     ISL|      Iceland|2020-03-31|      0.003326007326007326|
|     ESP|        Spain|2020-03-31|      0.002051618975538...|
|     CHE|  Switzerland|2020-03-31|      0.001918628875358...|
|     LIE|Liechtenstein|2020-03-31|      0.001783045336549...|
|     ITA|        Italy|2020-03-31|      0.001749732078891...|
|     MCO|       Monaco|2020-03-31|      0.001325043318723...|
|     AUT|      Austria|2020-03-31|      0.001130307337004...|
|     BEL|      Belgium|2020-03-31|      0.001102279833947...|
|     DEU|      Germany|2020-03-31|      8.570615766540

### Задние 2.Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [10]:
# Фильтруем записи по условию поподанию в требуемый временной промежуток
df_in_time_range = df_clean.select(
    ['iso_code',
    'location',
    'date',
    'new_cases']).filter( 
        (df['date'] > '2020-03-23') & (df['date'] < '2020-04-01'))

df_in_time_range.show()

+--------+-----------+----------+---------+
|iso_code|   location|      date|new_cases|
+--------+-----------+----------+---------+
|     AFG|Afghanistan|2020-03-24|      2.0|
|     AFG|Afghanistan|2020-03-25|     33.0|
|     AFG|Afghanistan|2020-03-26|      4.0|
|     AFG|Afghanistan|2020-03-27|     11.0|
|     AFG|Afghanistan|2020-03-28|     16.0|
|     AFG|Afghanistan|2020-03-29|     11.0|
|     AFG|Afghanistan|2020-03-30|     28.0|
|     AFG|Afghanistan|2020-03-31|     29.0|
|     ALB|    Albania|2020-03-24|     19.0|
|     ALB|    Albania|2020-03-25|     23.0|
|     ALB|    Albania|2020-03-26|     28.0|
|     ALB|    Albania|2020-03-27|     12.0|
|     ALB|    Albania|2020-03-28|     11.0|
|     ALB|    Albania|2020-03-29|     15.0|
|     ALB|    Albania|2020-03-30|     11.0|
|     ALB|    Albania|2020-03-31|     20.0|
|     DZA|    Algeria|2020-03-24|     34.0|
|     DZA|    Algeria|2020-03-25|     38.0|
|     DZA|    Algeria|2020-03-26|     65.0|
|     DZA|    Algeria|2020-03-27

In [17]:
# Группируем по странам и считаем сумму случаев 
df_sum_new_cases_per_time = df_in_time_range.groupby('iso_code').sum()

In [19]:
# Выводим TOP10 стран по количеству новых случаев
df_sum_new_cases_per_time.orderBy('sum(new_cases)', ascending=False).show(10)

+--------+--------------+
|iso_code|sum(new_cases)|
+--------+--------------+
|     USA|      146165.0|
|     ESP|       60787.0|
|     DEU|       42752.0|
|     ITA|       41865.0|
|     FRA|       32155.0|
|     GBR|       26071.0|
|     IRN|       21556.0|
|     TUR|       12002.0|
|     BEL|        9032.0|
|     NLD|        7903.0|
+--------+--------------+
only showing top 10 rows



### Задние 3. Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [26]:
import pyspark.sql.functions as f
from pyspark.sql import Window

In [25]:
df_in_time_range_RUS = df_in_time_range.filter(df_in_time_range['iso_code'] == 'RUS')
df_in_time_range_RUS.show()

+--------+--------+----------+---------+
|iso_code|location|      date|new_cases|
+--------+--------+----------+---------+
|     RUS|  Russia|2020-03-24|     57.0|
|     RUS|  Russia|2020-03-25|    163.0|
|     RUS|  Russia|2020-03-26|    182.0|
|     RUS|  Russia|2020-03-27|    196.0|
|     RUS|  Russia|2020-03-28|    228.0|
|     RUS|  Russia|2020-03-29|    270.0|
|     RUS|  Russia|2020-03-30|    302.0|
|     RUS|  Russia|2020-03-31|    501.0|
+--------+--------+----------+---------+



In [30]:
w = Window.partitionBy('iso_code', 'location').orderBy('date') 
df_in_time_range_RUS.withColumn('lead', f.lag('new_cases', 1).over(w)) \
  .withColumn('new_cases_diff', f.when(f.col('lead').isNotNull(), f.col('new_cases') - f.col('lead')).otherwise(f.lit(None))) \
  .show(10, False)

+--------+--------+----------+---------+-----+--------------+
|iso_code|location|date      |new_cases|lead |new_cases_diff|
+--------+--------+----------+---------+-----+--------------+
|RUS     |Russia  |2020-03-24|57.0     |null |null          |
|RUS     |Russia  |2020-03-25|163.0    |57.0 |106.0         |
|RUS     |Russia  |2020-03-26|182.0    |163.0|19.0          |
|RUS     |Russia  |2020-03-27|196.0    |182.0|14.0          |
|RUS     |Russia  |2020-03-28|228.0    |196.0|32.0          |
|RUS     |Russia  |2020-03-29|270.0    |228.0|42.0          |
|RUS     |Russia  |2020-03-30|302.0    |270.0|32.0          |
|RUS     |Russia  |2020-03-31|501.0    |302.0|199.0         |
+--------+--------+----------+---------+-----+--------------+

